# Paths in a maze
Many common search problems in computer science can be treated as searching paths within a maze.
Aim of this assignment is devising a proper data structure to model a maze and support various path finding algorithms.
For the purpose of this exercise, the maze will be represented as a two-dimensional grid of cells.
A maze can have both *free* and *blocked* cells. Only free cells can be traversed.
You are free to decide whether cell adjacency applies only to the four main direction or if you want to support also diagonal adjacency.
A path in the maze is a sequence of adjacent traversable cells, starting from a given start position and ending at a given target position.

## Generating a maze
The `Maze` class should keep track of the grid representing its state.
It should also have instance attributes for the number of rows, number of columns, start and goal location. 
The maze should provide an initialization method to fill it with random blocked cells. The resulting maze should be fairly sparse in order to (almost always) guarantee the existence of a path between two positions. The sparseness can be treated as an additional attribute of an instance.

## Helper methods
It can be useful to have a function that checks whether the visit reached the goal during the search. Regardless of the visit strategy adopted, this is a check that all the algorithms will have to make.

In order to model the act of moving across the maze, it could be useful to define an appropriate method that given a location will return all the available *next* locations reachable from such a position. In this way, it is possible to decouple movement rules from the chosen visit strategies.

These are just suggestion. You may implement any additional helper method you deem suitable, or none at all.

## Visit strategies
The maze should support some of the most common visit strategies. It should support at least BFS and DFS visits. In the following subsection there will be a brief description of various well-known algorithms, but you are free to include any additional one you may wish to experiment with.
The A* search may be a bit trickier to implement correctly if you are unfamiliar with the topic, but it is objectively the most interesting one to play with.

### Depth-first search (DFS)

A [depth-first search](https://en.wikipedia.org/wiki/Depth-first_search) (DFS) is a search that goes as deeply as it can
before backtracking to its last decision point if it reaches a dead end.

The DFS algorithm, in its iterative version, usually relies on a last-in-first-out data structure known as a stack. 
Informally, in a stack the last inserted *item* is placed on top of the stack (`push`) and is the first to be pulled off the stack (`pop`). 
Note that implementing a stack using a Python list is as simple as always appending items onto its right end and always removing items from the same end. 

An in-progress depth-first search needs to keep track of two data structures: the stack of states that we are considering searching and the set of states that we have already searched. As long as there are more states to visit, the DFS will keep checking
whether the current state is a goal and adding its reachable set to the stack. It will also mark each state that has already
been searched as explored to avoid revisiting already seen positions. If the stack is empty, it means there is no state left to search.

If you are familiar with recursion, the DFS has a very natural recursive implementation that does not require a stack.

### Breadth-first search (BFS)
A [breadth-first search](https://en.wikipedia.org/wiki/Breadth-first_search) (BFS) is a search that locally explore the whole neighbourhood at a given depth before moving on. 

Be reminded that a complete implementation of BFS has been already discussed during the last training session of Module 2.

To implement BFS, a first-in-first-out data structure, a queue, is required.
Informally, in a queue the first *item* who got queued is the first one to be dequeued. At a minimum, a queue has
the same `push` and `pop` methods as a stack, and it can rely on the Python `deque` built-in to be implemented.

The BFS algorithm for a breadth-first search is identical to the iterative algorithm for DFS, once the stack is swapped with a queue. Such a swap changes the order in which states are searched and ensures that the states closest to the start state are searched first.

### A* 
[A*](https://en.wikipedia.org/wiki/A*_search_algorithm) (read, a-star) is an optimized search algorithm that aims at finding the shortest path from start state to goal state. A* uses a combination of a cost function and a heuristic function to focus its search on pathways most likely to get to the goal quickly.

The cost function, $g(n)$, evaluates the cost to get to a particular state. For a maze, it would be the number of steps needed to reach a given position. The heuristic function, $h(n)$, gives an estimate of the cost to get from the *current* position to the goal state. In order for the algorithm to produce an optimal solution, the heuristic must be admissible. An admissible heuristic is one that never overestimates the cost to reach the goal. In the case of a two-dimensional plane, one example is a straight-line distance heuristic: a straight line is always the shortest path!

The total cost for any state considered is $f(n) = g(n) + h(n)$. When choosing the next state to explore, A* picks the one with the lowest $f(n)$.

To pick the state with the lowest $f(n)$ you can uses a priority queue. A priority queue keeps its elements in an internal
order, such that the first element popped out is always the highest-priority element.
In the case of A*, the highest-priority item is the one with the lowest $f(n)$.
Python standard library contains `heappush` and `heappop` functions that will manage a list as if it was a binary heap.

#### Heuristics
A heuristic is an *intuition* about a way to solve a given problem.
In this scenario, a heuristic aims to choose the best location to search next in order to reach the goal *efficiently*.

In a way, it is but an educated guess, taking into account the expected *closeness* to the goal, on which move to make next.

Heuristics that calculate smaller values end up leading to a search through more states, whereas heuristics closer to the
exact real distance, without overshooting otherwise they would be inadmissible, lead to a search through fewer states. 
The better the estimate, the more performing the heuristic. 

- As geometry teaches us, the shortest path between two points is a straight line. A straight-line heuristic will always be admissible for our search. The Euclidean distance, derived from the Pythagorean theorem, can be used directly by A* in order to guesstimate the distance from the goal
- If you are supporting movements only along the four main directions, there is a better heuristic you may wish to adopt: the Manhattan distance

## (Optional) Search visualization
The `pygame` library is a cross-platform set of Python modules designed for writing video games. In its most basic usage, it can be used to produce simple 2d visualization tools. You can try to integrate it in your solution to visualize the steps taken from the various visits your maze supports.

Note that this is a far more advanced request, so it is <u>completely optional</u>.

In [1]:
import numpy as np
import pygame
from queue import PriorityQueue
from collections import deque

WIDTH = 800
HEIGHT= 800
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Labyrinth algorithm")
pygame.init()

WHITE = (255, 255, 255)
GREY = (128, 128, 128)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)



class Point():
    def __init__(self,x:int,y:int):
        self.x=x
        self.y=y

    def __eq__(self,other):
        return self.x==other.x and self.y==other.y
    
    def __str__(self):
        return f"X= {self.x} ; Y= {self.y}"

class Maze():
    def __init__(self,n_row,n_col,start:Point,end:Point):
        self.n_row=n_row
        self.n_col=n_col
        self.start=start
        self.end=end
        self.maze=np.random.randint(2,size=(self.n_row,self.n_col))
        self.maze[start.x][start.y]=0
        self.maze[end.x][end.y]=0
        self.discoverd=list()
        self.min_path=list()

    def isTheExit(self,current:Point):
        return current==self.end

    def isPointValid(self,x:int,y:int):
        return ( 0 <= x < self.n_row ) and ( 0 <= y < self.n_col ) 

    def findPositionAround(self,currentPoint:Point):
        i=currentPoint.x
        j=currentPoint.y
        pointAround=list()
        pointAround.append(Point(i, j-1) if self.isPointValid(i, j-1) else None)
        pointAround.append(Point(i, j+1) if self.isPointValid(i, j+1) else None)

        pointAround.append(Point(i-1,j) if self.isPointValid(i-1,j) else None)
        pointAround.append(Point(i+1, j) if self.isPointValid(i+1, j) else None)

        pointAround.append(Point(i-1, j+1) if self.isPointValid(i-1, j+1) else None)
        pointAround.append(Point(i-1, j-1) if self.isPointValid(i-1, j-1) else None)

        pointAround.append(Point(i+1, j+1) if self.isPointValid(i+1, j+1) else None)
        pointAround.append(Point(i+1, j-1) if self.isPointValid(i+1, j-1) else None)
        return list(filter(lambda e: e is not None and ( self.maze[e.x][e.y]==0 ),pointAround))
        
    def DFS(self): 
        self.discoverd.clear()

        self.gridcolor=drawGrid(self.maze)
        updateColor(self.gridcolor,GREEN,self.start.x,self.start.y)
        updateColor(self.gridcolor,RED,self.end.x,self.end.y)


        return self._DFS(self.start)

    def _DFS(self,current:Point):

        if self.end == current:
            print("Trovato e il path è:",len(self.discoverd))
            updateColor(self.gridcolor,RED,current.x,current.y)
        else:
            updateColor(self.gridcolor,GREEN,current.x,current.y)

        self.discoverd.append(current)
        #print("I'm on = ",current)

        for v in self.findPositionAround(current):
            #print(f"{current} is and closed to {v}")
            if v not in self.discoverd:
                self._DFS(v)

    def BFS(self):
        q=deque()
        q.append(self.start)
        self.discoverd.clear()
        self.gridcolor=drawGrid(self.maze)
        updateColor(self.gridcolor,GREEN,self.start.x,self.start.y)
        updateColor(self.gridcolor,RED,self.end.x,self.end.y)
        return self._BFS(q)

    def _BFS(self,q:deque):

        if not q:
            return

        current=q.popleft()
        self.discoverd.append(current)
        #print("I'm on = ",current)

        if self.end == current:
            #print("Trovato")
            updateColor(self.gridcolor,RED,current.x,current.y)
        else:
            updateColor(self.gridcolor,GREEN,current.x,current.y)

        
        for v in self.findPositionAround(current):
            if v not in self.discoverd:
                #print(f"{current} has discovered {v}")
                self.discoverd.append(v)
                updateColor(self.gridcolor,GREEN,v.x,v.y)
                q.append(v)                
        self._BFS(q)

    def A_Star(self):
        q=PriorityQueue()
        q.put((1,self.start))
        self.discoverd.clear()

        self.gridcolor=drawGrid(self.maze)
        updateColor(self.gridcolor,GREEN,self.start.x,self.start.y)
        updateColor(self.gridcolor,RED,self.end.x,self.end.y)

        #funzione costo
        self.h_x=lambda i,j : abs(self.end.x - i) + abs(self.end.y - j)
        self.finish=False
        #collezione di path
        path=list()
        self._A_Star(q,path)
        
        return 

    def _A_Star(self,q:PriorityQueue,path:list):

        if q.empty():
            return

        if self.finish == True:
            return
        current=q.get()[1]
        self.discoverd.append(current)
        #print("I'm on = ",current," h(x)= ",self.h_x(current.x,current.y))
        path.append(str(current))

        if self.end == current:
            updateColor(self.gridcolor,RED,current.x,current.y)
            self.finish=True
            return
        else:
            updateColor(self.gridcolor,GREEN,current.x,current.y)

        for v in self.findPositionAround(current):
            if v not in self.discoverd:
                print(f"{current} has discovered {v} h(x)={self.h_x(v.x,v.y)} ")
                self.discoverd.append(v)
                updateColor(self.gridcolor,GREEN,v.x,v.y)
                q.put((int(self.h_x(v.x,v.y)),v))     

        #q=deque(sorted(q,key=lambda a: self.h_x(a.x,a.y)))
        self._A_Star(q,path)

class Cell:
    def __init__(self,color, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = color
        self.width = width
        self.total_rows = total_rows

    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y,self.width ,self.width))

def updateColor(gridcolor,color,x,y):
    s=gridcolor[y][x]
    s.color=color
    s.draw(WIN)
    #pygame.time.wait(50)
    pygame.display.update()

def colorCell(matrix:np,x:int,y:int):
    if matrix[x][y] == 0:
        return WHITE
    else:
        return BLACK 

def drawGrid(matrix:np):
    x,y= matrix.shape
    WIN.fill(WHITE)
    gap=HEIGHT//x
    grid=[]
    
    for r in range(x):
        grid.append([])
        for c in range(y):
            grid[r].append(Cell(colorCell(matrix,c,r),r,c,gap,r))
    
    for i in range(x):
        pygame.draw.line(WIN, GREY, (0, i * gap), (WIDTH, i * gap))
        for j in range(y):
            pygame.draw.line(WIN, GREY, (j * gap, 0), (j * gap, WIDTH))

    for row in grid:
        for spot in row:
            spot.draw(WIN)
    pygame.display.update()
    return grid

def main():
    ## definizione punti di ingresso punti di arrivo:
    ## LA definizione dei punti di inizio e di fine è lasciata al utente
    start   =Point(0,0)
    end     =Point(79,70)
    m=Maze(80,80,start,end)

    #La matrice è genrata automaticmente ( non c'è certezza che si riesca a ragiungere la destinazione )
    #Data la generazione automatica , il punto di arrivo potrebbe essere isolato
    #Il programma termina automaticamente nei tre algoritmi
    # I programmi espolarono tutto lo spazio delle soluzioni. Il terzo si ferma al raggiugimento dell punto di arrivo

    #m.DFS()        # print DFS 
    #m.BFS()        # print BFS
    m.A_Star()     # print A star algo 

    done=False
    while not done:  
        for event in pygame.event.get():  
            if event.type == pygame.QUIT: 
                done=True 
    pygame.quit()

if __name__ == "__main__":
    main()

pygame 2.1.2 (SDL 2.0.18, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


AttributeError: 'tuple' object has no attribute 'x'